<a href="https://colab.research.google.com/github/likuoping136/stable-diffusion-webui-colab/blob/main/install_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

%env PYTHONDONTWRITEBYTECODE=1

# 导入必要的模块用于检查路径存在性
import os

# 检查目标目录是否存在，如果不存在则执行克隆
target_dir = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui'
if not os.path.exists(target_dir):
    !git clone https://github.com/likuoping136/stable-diffusion-webui.git /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui
    print("Stable Diffusion WebUI 克隆完成。")
else:
    print("Stable Diffusion WebUI 目录已存在，跳过克隆操作。")

# 检查目标文件是否存在，如果不存在则下载
file_path = '/content/libtcmalloc_minimal.so.4'
if not os.path.exists(file_path):
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4
    print("libtcmalloc_minimal.so.4 下载完成。")
else:
    print("libtcmalloc_minimal.so.4 文件已存在，跳过下载操作。")

%env LD_PRELOAD=/content/drive/MyDrive/stable-diffusion-webui-colab/
print("安装流程完成。")
